## Sanity Check for the merges!

> We have merged all the files (vehicles, delays and weather). Let's check wheather the merges are done with correct mapping and has the values that are supposed to be there!

In [1]:
# notebooks/01_load_week.py
import pandas as pd
from pathlib import Path

HIST = Path("data_hist")
WEEK = ["2025-07-14","2025-07-15","2025-07-16",
        "2025-07-17","2025-07-18","2025-07-19","2025-07-20"]

files = [HIST/d/"bus_minute.parquet" for d in WEEK]
week_df = pd.concat([pd.read_parquet(f) for f in files])

print(f"rows : {len(week_df):,}")
print("period:", week_df.datetime_utc.min(), "→", week_df.datetime_utc.max())
week_df.to_parquet("week_2025‑07‑14_20.parquet",
                   engine="pyarrow", compression="zstd")


rows : 5,033,195
period: 2025-07-13 22:43:12+00:00 → 2025-07-20 22:59:38+00:00


In [2]:
# % of valid speeds
week_df["speed_ok"] = week_df.speed_kph.between(0.5, 120)
print("share with valid speed:",
      week_df.speed_ok.mean().round(3))

# extreme delays
late = week_df.delay_sec > 900         # > 15 min late
print("rows >15 min late:", late.sum())

# route 41 inbound median speed by hour‑of‑day
sel   = week_df.query("route_id == '4700_102333' and direction_id == 1")
sel["hour"] = sel.datetime_utc.dt.tz_convert("Europe/Dublin").dt.hour
print(sel.groupby("hour").speed_kph.median())


share with valid speed: 0.759
rows >15 min late: 1285
Series([], Name: speed_kph, dtype: float64)


In [8]:
import pandas as pd

In [9]:
week_df = pd.read_parquet('../data_hist/week_2025‑07‑14_20.parquet')

In [11]:
week_df


,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,zone,minute,temp_c,winds_kph,rain_mm,wcode,datetime_ie
0,5438,4707_4215,4702_101125,0,1752446592,2025-07-13 22:43:12+00:00,53.306946,-6.334197,0.000000,NaN,south,2025-07-13 22:43:00,NaN,NaN,NaN,NaN,2025-07-13 23:43:12
1,5438,4707_4215,4702_101125,0,1752446592,2025-07-13 22:43:12+00:00,53.306946,-6.334197,0.000000,NaN,south,2025-07-13 22:43:00,NaN,NaN,NaN,NaN,2025-07-13 23:43:12
2,5438,4707_4215,4702_101125,0,1752446592,2025-07-13 22:43:12+00:00,53.306946,-6.334197,NaN,NaN,south,2025-07-13 22:43:00,NaN,NaN,NaN,NaN,2025-07-13 23:43:12
3,5339,4707_6104,4702_101049,0,1752446642,2025-07-13 22:44:02+00:00,53.279289,-6.402237,NaN,NaN,south,2025-07-13 22:44:00,NaN,NaN,NaN,NaN,2025-07-13 23:44:02
4,5339,4707_6104,4702_101049,0,1752446642,2025-07-13 22:44:02+00:00,53.279289,-6.402237,0.000000,NaN,south,2025-07-13 22:44:00,NaN,NaN,NaN,NaN,2025-07-13 23:44:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146636,1289,4710_4210,4710_102349,0,1753052352,2025-07-20 22:59:12+00:00,53.460400,-6.352144,74.647795,NaN,north,2025-07-20 22:59:00,15.9,4.3,0.0,53.0,2025-07-20 23:59:12
146637,1152,4710_75407,4710_102337,0,1753052357,2025-07-20 22:59:17+00:00,54.426380,-6.984912,72.016638,NaN,north,2025-07-20 22:59:00,15.9,4.3,0.0,53.0,2025-07-20 23:59:17
146638,1842,4710_56750,4710_102335,0,1753052351,2025-07-20 22:59:11+00:00,54.265804,-8.481790,118.349735,NaN,north,2025-07-20 22:59:00,15.9,4.3,0.0,53.0,2025-07-20 23:59:11
146639,1897,4710_7424,4710_102354,0,1753052357,2025-07-20 22:59:17+00:00,53.363316,-6.303486,38.597001,NaN,north,2025-07-20 22:59:00,15.9,4.3,0.0,53.0,2025-07-20 23:59:17


In [4]:
import pandas as pd
import polars as pl
from pathlib import Path
stops = pl.read_csv("../gtfs_static/stop_times.txt")[["trip_id","stop_sequence","stop_id","arrival_time"]]
term = stops(["trip_id","stop_sequence"]).group_by("trip_id").tail(1)

In [5]:
term

trip_id,stop_sequence,stop_id,arrival_time
str,i64,str,str
"""3026_1""",6,"""8300B1070101""","""08:45:00"""
"""3026_2""",6,"""8310B1046501""","""14:15:00"""
"""3026_3""",6,"""8310B1046501""","""18:15:00"""
"""3094_1""",17,"""826000173""","""08:45:00"""
"""3094_2""",17,"""826000173""","""08:45:00"""
…,…,…,…
"""4727_9968""",10,"""8260IR0063""","""22:20:00"""
"""4727_9977""",12,"""8280IR0068""","""23:21:00"""
"""4727_9978""",10,"""8260IR0063""","""23:20:00"""


## We did the merge of vehicles and delay fetches. Let's check if they merged properly!

In [12]:
df_14 = pd.read_parquet('../data_hist/2025-07-14/veh_minute_w.parquet')

In [19]:
df_14['delay_sec'][df_14['delay_sec']<=0]

749         0.0
1685       -1.0
1686      -56.0
1695      -73.0
2023        0.0
          ...  
826868   -108.0
826901      0.0
826913   -108.0
826947      0.0
826960   -108.0
Name: delay_sec, Length: 4644, dtype: float64

In [21]:
df_14_van = pd.read_parquet('../data_hist/2025-07-14/veh_minute_w_vanished.parquet')

In [22]:
df_14_van['delay_sec'][df_14_van['delay_sec']<=0]

10266      -78.0
10576     -511.0
10689     -158.0
10946     -245.0
10962     -142.0
           ...  
826482     -13.0
827007       0.0
827144     -46.0
827283    -929.0
827310   -1070.0
Name: delay_sec, Length: 4644, dtype: float64

In [23]:
df_delay = pd.read_parquet('../data_hist/2025-07-14/delay.parquet')

In [28]:
df_delay

,trip_id,route_id,direction_id,timestamp,delay_sec,datetime_utc
0,4710_75386,4710_102337,0,1752447631,4726,2025-07-13 23:00:31+00:00
1,4710_63213,4710_102336,0,1752447631,1554,2025-07-13 23:00:31+00:00
2,4710_56750,4710_102335,0,1752447631,2459,2025-07-13 23:00:31+00:00
3,4710_9671,4710_102545,0,1752447631,2102,2025-07-13 23:00:31+00:00
4,4710_75407,4710_102337,0,1752447631,423,2025-07-13 23:00:31+00:00
...,...,...,...,...,...,...
1737103,4707_13961,4707_101987,1,1752533974,0,2025-07-14 22:59:34+00:00
1737104,4700_847,4700_100865,0,1752533974,0,2025-07-14 22:59:34+00:00
1737105,4710_2174,4710_102348,0,1752533974,0,2025-07-14 22:59:34+00:00
1737106,4710_8440,4710_102509,0,1752533974,66,2025-07-14 22:59:34+00:00


In [29]:
df_delay["trip_id"].nunique()

11960

In [30]:
df_veh = pd.read_parquet('../data_hist/2025-07-14/vehicles.parquet')

In [31]:
df_veh

,vehicle_id,trip_id,direction_id,timestamp,lat,lon,datetime_utc,speed_kph
91003,1,4700_901,1,1752475561,53.156853,-6.910296,2025-07-14 06:46:01+00:00,NaN
91879,1,4700_901,1,1752475561,53.156853,-6.910296,2025-07-14 06:46:01+00:00,0.0
92760,1,4700_901,1,1752475561,53.156853,-6.910296,2025-07-14 06:46:01+00:00,0.0
93647,1,4700_901,1,1752475561,53.156853,-6.910296,2025-07-14 06:46:01+00:00,0.0
94536,1,4700_901,1,1752475812,53.156853,-6.910296,2025-07-14 06:50:12+00:00,0.0
...,...,...,...,...,...,...,...,...
618451,6736,4707_13769,0,1752512469,53.336300,-6.468938,2025-07-14 17:01:09+00:00,0.0
619445,6736,4707_13769,0,1752512469,53.336300,-6.468938,2025-07-14 17:01:09+00:00,0.0
620439,6736,4707_13769,0,1752512469,53.336300,-6.468938,2025-07-14 17:01:09+00:00,0.0
621434,6736,4707_13769,0,1752512469,53.336300,-6.468938,2025-07-14 17:01:09+00:00,0.0


In [34]:
df_veh["trip_id"].nunique()

10103

In [35]:
df_veh["vehicle_id"].nunique()

1284

In [36]:
df_14_van["trip_id"].nunique()

10103

In [89]:
df_delay[df_delay["direction_id"]==1][df_delay["trip_id"]=="4700_901"]

C:\Users\mkuma\AppData\Local\Temp\ipykernel_8304\4064625731.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_delay[df_delay["direction_id"]==1][df_delay["trip_id"]=="4700_901"]


,trip_id,route_id,direction_id,timestamp,delay_sec,datetime_utc
184483,4700_901,4700_100865,1,1752474146,0,2025-07-14 06:22:26+00:00
186153,4700_901,4700_100865,1,1752474208,0,2025-07-14 06:23:28+00:00
187828,4700_901,4700_100865,1,1752474270,0,2025-07-14 06:24:30+00:00
189514,4700_901,4700_100865,1,1752474329,0,2025-07-14 06:25:29+00:00
191209,4700_901,4700_100865,1,1752474397,0,2025-07-14 06:26:37+00:00
...,...,...,...,...,...,...
536636,4700_901,4700_100865,1,1752486037,-383,2025-07-14 09:40:37+00:00
538373,4700_901,4700_100865,1,1752486088,-383,2025-07-14 09:41:28+00:00
540114,4700_901,4700_100865,1,1752486157,-383,2025-07-14 09:42:37+00:00
541865,4700_901,4700_100865,1,1752486210,-383,2025-07-14 09:43:30+00:00


In [86]:
#df_14_van[df_14_van["trip_id"]=="4700_901"]
df_14_van[df_14_van["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][df_14_van["trip_id"]=="4700_901"]

C:\Users\mkuma\AppData\Local\Temp\ipykernel_8304\862645137.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_14_van[df_14_van["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][df_14_van["trip_id"]=="4700_901"]


,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,minute,temp_c,winds_kph,rain_mm,wcode,datetime_ie,sched_end_ts_utc,gap_forward_sec,vanish_anchor,vanished
180,1,4700_901,4700_100865,1,1752479167,2025-07-14 07:46:07+00:00,53.209667,-6.738799,27.920731,NaN,...,2025-07-14 07:46:00,16.8,11.5,0.0,3.0,2025-07-14 08:46:07,1.752485e+09,62.0,False,False


In [84]:
df_delay[df_delay["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][df_delay["trip_id"]=="4700_901"]

C:\Users\mkuma\AppData\Local\Temp\ipykernel_8304\3731685110.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_delay[df_delay["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][df_delay["trip_id"]=="4700_901"]


,trip_id,route_id,direction_id,timestamp,delay_sec,datetime_utc
338050,4700_901,4700_100865,1,1752479197,47,2025-07-14 07:46:37+00:00


In [85]:
df_veh[df_veh["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][df_veh["trip_id"]=="4700_901"]

C:\Users\mkuma\AppData\Local\Temp\ipykernel_8304\1298154671.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_veh[df_veh["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][df_veh["trip_id"]=="4700_901"]


,vehicle_id,trip_id,direction_id,timestamp,lat,lon,datetime_utc,speed_kph
148585,1,4700_901,1,1752479167,53.209667,-6.738799,2025-07-14 07:46:07+00:00,27.920731


In [88]:
print("rows :", len(df_14_van))
print("delay not-null share:", df_14_van["delay_sec"].notna().mean())

rows : 827334
delay not-null share: 0.012815863967877544


## **❌ Alert! ❌**
**This is bad. Out of almost a million values only 1.2% of the values had any numbers. That is just entirely bad as it suggests that there is a flaw in the logic of mapping** 
> The above EDA shows how the delay is available in delay.parquet(which is our per minute fetch) but is not available after the merge. 

# Major Update
> Upon checking the above files I found out that while the initial merge i did produced lousy output! What I mean by that is that the merge was done as per trip_id and timestamp. Now the issue was that the vehicle fetch and delay fetch were done at different seconds within the minute so the timestamp mostly didn't match due to the mismatch in seconds. I checked the merge and saw majorly NaN values. which was entirely a FAIL in mapping logic. So I merged the files as per the minute level in the timestamp and ignored the second. 

>**Below we can see that the result has improved with 96% of the mapped values has delays**
- I even checked the delay for the same trip and time (only hour and minute). The previous map showed NaN however now I can see the exact delay in the new mapped file as shown in the delay fetch!

# We have updated Veh_minute.parquet! Let's Check!

In [69]:
veh_minute_df = pd.read_parquet('../data_hist/2025-07-14/veh_minute.parquet')

In [70]:
veh_minute_df

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec
0,1,4700_901,4700_100865,1,1752475561,2025-07-14 06:46:01+00:00,53.156853,-6.910296,NaN,0.0
1,1,4700_901,4700_100865,1,1752475561,2025-07-14 06:46:01+00:00,53.156853,-6.910296,0.0,0.0
2,1,4700_901,4700_100865,1,1752475561,2025-07-14 06:46:01+00:00,53.156853,-6.910296,0.0,0.0
3,1,4700_901,4700_100865,1,1752475561,2025-07-14 06:46:01+00:00,53.156853,-6.910296,0.0,0.0
4,1,4700_901,4700_100865,1,1752475812,2025-07-14 06:50:12+00:00,53.156853,-6.910296,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
827329,6736,4707_13769,4702_101110,0,1752512469,2025-07-14 17:01:09+00:00,53.336300,-6.468938,0.0,181.0
827330,6736,4707_13769,4702_101110,0,1752512469,2025-07-14 17:01:09+00:00,53.336300,-6.468938,0.0,181.0
827331,6736,4707_13769,4702_101110,0,1752512469,2025-07-14 17:01:09+00:00,53.336300,-6.468938,0.0,181.0
827332,6736,4707_13769,4702_101110,0,1752512469,2025-07-14 17:01:09+00:00,53.336300,-6.468938,0.0,181.0


In [74]:
veh_minute_df["delay_sec"][veh_minute_df["delay_sec"].isna()]

501      NaN
602      NaN
716      NaN
831      NaN
832      NaN
          ..
826801   NaN
827055   NaN
827174   NaN
827175   NaN
827176   NaN
Name: delay_sec, Length: 27252, dtype: float64

In [75]:
print("rows :", len(veh_minute_df))
print("delay not-null share:", veh_minute_df["delay_sec"].notna().mean())

rows : 827334
delay not-null share: 0.9670604616756957


In [87]:
veh_minute_df[veh_minute_df["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][veh_minute_df["trip_id"]=="4700_901"]

C:\Users\mkuma\AppData\Local\Temp\ipykernel_8304\1982600162.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  veh_minute_df[veh_minute_df["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M") == "2025-07-14 07:46"][veh_minute_df["trip_id"]=="4700_901"]


,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec
60,1,4700_901,4700_100865,1,1752479167,2025-07-14 07:46:07+00:00,53.209667,-6.738799,27.920731,47.0


### Happy Coding Happy Analysis!
> We have exact delay match for the trip in that minute! happy days!

# Let's check another Dublin Shapes file

In [1]:
import pandas as pd

In [2]:
dub_shapes = pd.read_parquet('../gtfs_static/dublin_shapes.parquet')

In [3]:
dub_shapes

,shape_id,seq,lat,lon
0,3026_1,0,53.512881,-6.538782
1,3026_1,1,53.512898,-6.537984
2,3026_1,2,53.512914,-6.537818
3,3026_1,3,53.512940,-6.537727
4,3026_1,4,53.513011,-6.537634
...,...,...,...,...
3241350,4727_97,655,53.449243,-6.156835
3241351,4727_97,656,53.449431,-6.156807
3241352,4727_97,657,53.450178,-6.156693
3241353,4727_97,658,53.450344,-6.156665


In [4]:
import pandas as pd
shapes = pd.read_parquet("../gtfs_static/dublin_shapes.parquet")
print(shapes.columns.tolist())
print(shapes.head())


['shape_id', 'seq', 'lat', 'lon']
  shape_id  seq        lat       lon
0   3026_1    0  53.512881 -6.538782
1   3026_1    1  53.512898 -6.537984
2   3026_1    2  53.512914 -6.537818
3   3026_1    3  53.512940 -6.537727
4   3026_1    4  53.513011 -6.537634


In [5]:
trips = pd.read_csv("../gtfs_static/dublin_trips.csv")
print(trips[["trip_id","shape_id"]].head())
print("missing shape_id rows:", trips["shape_id"].isna().sum())


  trip_id shape_id
0  3026_1   3026_1
1  3026_2   3026_2
2  3026_3   3026_2
3  3094_1   3094_1
4  3094_2   3094_1
missing shape_id rows: 0


# Checking the scored files

In [1]:
import pandas as pd

In [2]:
df_scored = pd.read_parquet('../data_hist/2025-07-20/norms/veh_minute_scored.parquet')

In [5]:
df_scored[df_scored["vanish_anchor"]==True]

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
38327,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,NaN,0.0,...,1900.0,True,True,0.0,42.914625,0.000000,0.0,0.00,0.0,0.100000
39314,1122,4710_3958,4710_102349,0,1753020255,2025-07-20 14:04:15+00:00,53.353802,-6.258412,4.961252,NaN,...,7513.0,True,True,0.0,63.650889,0.000000,0.0,0.04,0.0,0.103200
39583,1122,4710_4780,4710_102349,1,1753019463,2025-07-20 13:51:03+00:00,53.348301,-6.255154,8.571265,-4821.0,...,4024.0,True,True,0.0,64.115471,0.866315,0.0,0.02,0.0,0.361495
41648,1152,4710_75239,4710_102337,0,1753000028,2025-07-20 08:27:08+00:00,53.348949,-6.251678,0.000000,0.0,...,2207.0,True,True,0.0,100.432485,0.000000,0.0,0.00,0.0,0.100000
43863,1159,4710_4111,4710_102349,0,1753034748,2025-07-20 18:05:48+00:00,53.354645,-6.275030,0.000000,-5559.0,...,3059.0,True,True,0.0,63.650889,0.000000,0.0,0.00,0.0,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127628,3245,4700_340,4700_100863,1,1753021211,2025-07-20 14:20:11+00:00,53.344612,-7.041933,11.371739,0.0,...,3790.0,True,True,0.0,79.465707,0.856898,0.0,0.05,0.0,0.361069
129134,3248,4700_938,4700_100865,1,1753001625,2025-07-20 08:53:45+00:00,53.156853,-6.910296,0.000000,0.0,...,1804.0,True,True,0.0,84.852064,0.000000,0.0,0.00,0.0,0.100000
132283,5320,4710_12140,4710_102356,1,1753007359,2025-07-20 10:29:19+00:00,53.515415,-7.343437,0.000000,-5082.0,...,2602.0,True,True,0.0,97.342402,0.000000,0.0,0.00,0.0,0.100000
135916,5361,4710_12295,4710_102356,1,1753011560,2025-07-20 11:39:20+00:00,53.523632,-7.347239,0.000000,NaN,...,12892.0,True,True,0.0,97.342402,0.000000,0.0,0.00,0.0,0.100000


In [6]:
df_scored[df_scored["vanished"]==True]

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
38311,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000,42.914625,0.000000,0.0,0.0,0.0,0.100000
38312,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000,42.914625,0.000000,0.0,0.0,0.0,0.100000
38313,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000,42.914625,0.000000,0.0,0.0,0.0,0.100000
38314,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000,42.914625,0.000000,0.0,0.0,0.0,0.100000
38315,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000,42.914625,0.000000,0.0,0.0,0.0,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143127,6669,4710_59825,4710_102544,0,1753029483,2025-07-20 16:38:03+00:00,51.900272,-8.463326,28.318141,185.0,...,60.0,False,True,0.185,100.726126,0.718860,0.0,0.0,0.0,0.389658
143128,6669,4710_59825,4710_102544,0,1753029543,2025-07-20 16:39:03+00:00,51.900528,-8.465281,8.222979,185.0,...,61.0,False,True,0.185,100.726126,0.918363,0.0,0.0,0.0,0.449509
143129,6669,4710_59825,4710_102544,0,1753029604,2025-07-20 16:40:04+00:00,51.900013,-8.465716,3.812518,185.0,...,61.0,False,True,0.185,100.726126,0.000000,0.0,0.0,0.0,0.174000
143130,6669,4710_59825,4710_102544,0,1753029665,2025-07-20 16:41:05+00:00,51.898781,-8.467019,9.654286,185.0,...,68.0,False,True,0.185,100.726126,0.904153,0.0,0.0,0.0,0.445246


In [15]:
# 1105	4707_19334
df_scored[df_scored["trip_id"]=="4707_19334"][df_scored["vehicle_id"]==1105][df_scored["vanished"]==True]


C:\Users\mkuma\AppData\Local\Temp\ipykernel_30008\3385402996.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_scored[df_scored["trip_id"]=="4707_19334"][df_scored["vehicle_id"]==1105][df_scored["vanished"]==True]
C:\Users\mkuma\AppData\Local\Temp\ipykernel_30008\3385402996.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_scored[df_scored["trip_id"]=="4707_19334"][df_scored["vehicle_id"]==1105][df_scored["vanished"]==True]


,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
38311,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000000,42.914625,0.0,0.0,0.0,0.0,0.100000
38312,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000000,42.914625,0.0,0.0,0.0,0.0,0.100000
38313,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000000,42.914625,0.0,0.0,0.0,0.0,0.100000
38314,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000000,42.914625,0.0,0.0,0.0,0.0,0.100000
38315,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.000000,0.0,...,0.0,False,True,0.000000,42.914625,0.0,0.0,0.0,0.0,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38389,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38390,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38391,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.232362,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38392,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933


In [16]:
df_scored[df_scored["trip_id"]=="4707_19334"][df_scored["vehicle_id"]==1105][df_scored["vanish_anchor"]==True]

C:\Users\mkuma\AppData\Local\Temp\ipykernel_30008\3468960208.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_scored[df_scored["trip_id"]=="4707_19334"][df_scored["vehicle_id"]==1105][df_scored["vanish_anchor"]==True]
C:\Users\mkuma\AppData\Local\Temp\ipykernel_30008\3468960208.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_scored[df_scored["trip_id"]=="4707_19334"][df_scored["vehicle_id"]==1105][df_scored["vanish_anchor"]==True]


,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
38327,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,NaN,0.0,...,1900.0,True,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1


In [17]:
df_scored.to_csv("../data_hist/2025-07-20/norms/veh_minute_scored.csv")

In [19]:
t = df_scored.query("vanish_anchor").iloc[0]["trip_id"]
display(df_scored[df_scored.trip_id == t].head(10))     # beginning of trip
display(df_scored[df_scored.trip_id == t].tail(10))     # end of trip

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
38311,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38312,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38313,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38314,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38315,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38316,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38317,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38318,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38319,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1
38320,1105,4707_19334,4702_101123,1,1752997604,2025-07-20 07:46:44+00:00,53.394176,-6.391766,0.0,0.0,...,0.0,False,True,0.0,42.914625,0.0,0.0,0.0,0.0,0.1


,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
38384,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38385,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38386,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38387,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38388,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38389,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38390,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38391,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.232362,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38392,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,0.0,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933
38393,1105,4707_19334,4702_101123,1,1753002347,2025-07-20 09:05:47+00:00,53.380123,-6.235768,0.000000,401.0,...,NaN,False,True,0.367333,42.914625,0.0,0.0,0.0,0.0,0.246933


In [22]:
import pandas as pd
df = pd.read_parquet("../data_hist/2025-07-20/norms/veh_minute_scored.parquet",
                     columns=["trip_id", "minute", "vanished","vanish_anchor"])

# trips with any vanished=True
bad_trips = df.query("vanished").trip_id.unique()[:5]     # take 5 samples

for t in bad_trips:
    sample = df[df.trip_id == t]
    print(t, "first 3 rows →")
    print(sample.head(3)[["minute","vanished","vanish_anchor"]])



4707_19334 first 3 rows →
                   minute  vanished  vanish_anchor
38311 2025-07-20 07:46:00      True          False
38312 2025-07-20 07:46:00      True          False
38313 2025-07-20 07:46:00      True          False
4710_3958 first 3 rows →
                   minute  vanished  vanish_anchor
39302 2025-07-20 13:51:00      True          False
39303 2025-07-20 13:51:00      True          False
39304 2025-07-20 13:53:00      True          False
4710_4780 first 3 rows →
                   minute  vanished  vanish_anchor
39577 2025-07-20 13:45:00      True          False
39578 2025-07-20 13:46:00      True          False
39579 2025-07-20 13:46:00      True          False
4710_75239 first 3 rows →
                   minute  vanished  vanish_anchor
41571 2025-07-20 07:25:00      True          False
41572 2025-07-20 07:25:00      True          False
41573 2025-07-20 07:25:00      True          False
4710_4111 first 3 rows →
                   minute  vanished  vanish_anchor
43831 

In [24]:
for t in bad_trips:
    print("last 3 rows →")
    print(sample.tail(3)[["minute","vanished","vanish_anchor"]], "\n")

last 3 rows →
                   minute  vanished  vanish_anchor
43952 2025-07-20 20:28:00      True          False
43953 2025-07-20 20:29:00      True          False
43954 2025-07-20 20:30:00      True          False 

last 3 rows →
                   minute  vanished  vanish_anchor
43952 2025-07-20 20:28:00      True          False
43953 2025-07-20 20:29:00      True          False
43954 2025-07-20 20:30:00      True          False 

last 3 rows →
                   minute  vanished  vanish_anchor
43952 2025-07-20 20:28:00      True          False
43953 2025-07-20 20:29:00      True          False
43954 2025-07-20 20:30:00      True          False 

last 3 rows →
                   minute  vanished  vanish_anchor
43952 2025-07-20 20:28:00      True          False
43953 2025-07-20 20:29:00      True          False
43954 2025-07-20 20:30:00      True          False 

last 3 rows →
                   minute  vanished  vanish_anchor
43952 2025-07-20 20:28:00      True          False
4395

# ANother Sanity Check

In [25]:
import pandas as pd

In [26]:
scored_df = pd.read_parquet('../data_hist/2025-07-14/norms/veh_minute_scored.parquet')

In [27]:
scored_df

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress
0,1,4700_745,4700_100865,0,1752506581,2025-07-14 15:23:01+00:00,53.350243,-6.250592,0.054333,180.0,...,58.0,False,False,0.18,84.880629,0.000000,0.0,0.0,0.0,0.072000
1,1,4700_745,4700_100865,0,1752506639,2025-07-14 15:23:59+00:00,53.349335,-6.251461,7.216326,180.0,...,62.0,False,False,0.18,84.880629,0.914983,0.0,0.0,0.0,0.300746
2,1,4700_745,4700_100865,0,1752506701,2025-07-14 15:25:01+00:00,53.347328,-6.252004,13.123150,180.0,...,64.0,False,False,0.18,84.880629,0.845393,0.0,0.0,0.0,0.283348
3,1,4700_745,4700_100865,0,1752506765,2025-07-14 15:26:05+00:00,53.347397,-6.253308,4.886600,180.0,...,66.0,False,False,0.18,84.880629,0.000000,0.0,0.0,0.0,0.072000
4,1,4700_745,4700_100865,0,1752506831,2025-07-14 15:27:11+00:00,53.347397,-6.254719,5.108637,180.0,...,53.0,False,False,0.18,84.880629,0.939814,0.0,0.0,0.0,0.306953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827329,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,0.0,False,False,0.00,57.553242,0.000000,0.0,0.0,0.0,0.000000
827330,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,0.0,False,False,0.00,57.553242,0.000000,0.0,0.0,0.0,0.000000
827331,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,0.0,False,False,0.00,57.553242,0.000000,0.0,0.0,0.0,0.000000
827332,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,0.0,False,False,0.00,57.553242,0.000000,0.0,0.0,0.0,0.000000


In [28]:
scored_df[scored_df["row_stress"]>0.7]

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,gap_forward_sec,vanish_anchor,vanished,delay_norm,free_kph,speed_norm,rain_norm,heat_norm,cold_norm,row_stress


In [29]:
dsw_df = pd.read_parquet('../data_hist/2025-07-14/norms/veh_minute_dsw.parquet')

In [30]:
dsw_df.to_csv('../data_hist/2025-07-14/norms/veh_minute_dsw.csv')

# Hmm Update!

In [32]:
segments_df = pd.read_parquet('../cache/segments.parquet')

In [39]:
segments_df

,segment_id,shape_id,seq,geometry
0,3026_1_0,3026_1,0,"b""\x01\x02\x00\x00\x00\x02\x00\x00\x00\xceu3\x..."
1,3026_1_1,3026_1,1,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xd9\xe0...
2,3026_1_2,3026_1,2,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xf3FX\x...
3,3026_1_3,3026_1,3,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xcbuP\x...
4,3026_1_4,3026_1,4,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00R\xe1ym\...
...,...,...,...,...
3238559,4727_97_654,4727_97,654,b'\x01\x02\x00\x00\x00\x02\x00\x00\x008kKx\x9d...
3238560,4727_97_655,4727_97,655,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\x9ak\x8...
3238561,4727_97_656,4727_97,656,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xfd\x1b...
3238562,4727_97_657,4727_97,657,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xc5\x0f...


In [2]:

import pandas as pd
from pathlib import Path

DATA_HIST = Path("../data_hist")          # adjust if your notebook sits elsewhere
FILE_NAME = "veh_minute_w_vanished.parquet"

# gather all sub-folders that have the output file
daily_parquets = sorted((f/FILE_NAME for f in DATA_HIST.iterdir() if (f/FILE_NAME).exists()))

summary = []

for pqt in daily_parquets:
    df = pd.read_parquet(pqt, columns=["vehicle_id","trip_id","vanished"])
    
    n_rows      = df["vanished"].sum()
    vanished_df = df.loc[df["vanished"]]

    n_trips     = len(vanished_df[["vehicle_id","trip_id"]].drop_duplicates())
    n_vehicles  = vanished_df["vehicle_id"].nunique()

    summary.append({
        "date"                 : pqt.parent.name,
        "N_min_rows_vanished"  : int(n_rows),
        "N_trips_vanished"     : n_trips,
        "N_vehicles_vanished"  : n_vehicles,
    })

pd.DataFrame(summary).set_index("date").sort_index()

,N_min_rows_vanished,N_trips_vanished,N_vehicles_vanished
date,,,
2025-07-14,11790,90,68
2025-07-15,9649,67,54
2025-07-16,9157,61,51
2025-07-17,11497,75,59
2025-07-18,10142,75,60
2025-07-19,7036,50,37
2025-07-20,9410,61,45


In [14]:
ed_df = pd.read_parquet('../data_hist/2025-07-14/veh_minute_w_vanished.parquet')

In [15]:
ed_df

,vehicle_id,trip_id,route_id,direction_id,timestamp,datetime_utc,lat,lon,speed_kph,delay_sec,...,minute,temp_c,winds_kph,rain_mm,wcode,datetime_ie,sched_end_ts_utc,gap_forward_sec,vanish_anchor,vanished
0,1,4700_745,4700_100865,0,1752506581,2025-07-14 15:23:01+00:00,53.350243,-6.250592,0.054333,180.0,...,2025-07-14 15:23:00,17.5,16.6,0.0,55.0,2025-07-14 16:23:01,1.752514e+09,58.0,False,False
1,1,4700_745,4700_100865,0,1752506639,2025-07-14 15:23:59+00:00,53.349335,-6.251461,7.216326,180.0,...,2025-07-14 15:23:00,17.5,16.6,0.0,55.0,2025-07-14 16:23:59,1.752514e+09,62.0,False,False
2,1,4700_745,4700_100865,0,1752506701,2025-07-14 15:25:01+00:00,53.347328,-6.252004,13.123150,180.0,...,2025-07-14 15:25:00,17.5,16.6,0.0,55.0,2025-07-14 16:25:01,1.752514e+09,64.0,False,False
3,1,4700_745,4700_100865,0,1752506765,2025-07-14 15:26:05+00:00,53.347397,-6.253308,4.886600,180.0,...,2025-07-14 15:26:00,17.5,16.6,0.0,55.0,2025-07-14 16:26:05,1.752514e+09,66.0,False,False
4,1,4700_745,4700_100865,0,1752506831,2025-07-14 15:27:11+00:00,53.347397,-6.254719,5.108637,180.0,...,2025-07-14 15:27:00,17.5,16.6,0.0,55.0,2025-07-14 16:27:11,1.752514e+09,53.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827329,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,2025-07-14 07:26:00,16.6,12.2,0.0,3.0,2025-07-14 08:26:40,1.752478e+09,0.0,False,False
827330,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,2025-07-14 07:26:00,16.6,12.2,0.0,3.0,2025-07-14 08:26:40,1.752478e+09,0.0,False,False
827331,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,2025-07-14 07:26:00,16.6,12.2,0.0,3.0,2025-07-14 08:26:40,1.752478e+09,0.0,False,False
827332,6736,4707_19957,4702_101106,1,1752478000,2025-07-14 07:26:40+00:00,53.305016,-6.216809,0.000000,-1070.0,...,2025-07-14 07:26:00,16.6,12.2,0.0,3.0,2025-07-14 08:26:40,1.752478e+09,0.0,False,False


In [12]:
import geopandas as gpd
ed_geojson = gpd.read_file('../gtfs_static/census/dublin_ed_trimmed.geojson')

In [13]:
ed_geojson

,OBJECTID,ED_GUID,ED_OFFICIAL,ED_ENGLISH,ED_GAEILGE,ED_ID_STR,ED_PART_COUNT,COUNTY_CODE,COUNTY_ENGLISH,COUNTY_GAEILGE,CSO_LEA,geometry
0,1502,2ae19629-1c9f-13a3-e055-000000000001,Y,SKERRIES,NA SCEIRÍ,267123,1,04,FINGAL,Baile Átha Cliath,BALBRIGGAN,"POLYGON ((-6.10572 53.58608, -6.10571 53.58607..."
1,1503,2ae19629-1ca0-13a3-e055-000000000001,Y,STILLORGAN-DEERPARK,STIGH LORGAN-PÁIRC NA BHFIANNA,267124,1,05,DUN LAOGHAIRE/RATHDOWN,Baile Átha Cliath,STILLORGAN,"POLYGON ((-6.20933 53.29051, -6.20892 53.29009..."
2,1504,2ae19629-1ca1-13a3-e055-000000000001,Y,STILLORGAN-KILMACUD,STIGH LORGAN-CILL MOCHUDA,267125,1,05,DUN LAOGHAIRE/RATHDOWN,Baile Átha Cliath,STILLORGAN,"POLYGON ((-6.20894 53.28908, -6.20865 53.28907..."
3,1505,2ae19629-1ca2-13a3-e055-000000000001,Y,STILLORGAN-LEOPARDSTOWN,STIGH LORGAN-BAILE NA LOBHAR,267126,1,05,DUN LAOGHAIRE/RATHDOWN,Baile Átha Cliath,STILLORGAN,"POLYGON ((-6.1992 53.27305, -6.20092 53.273, -..."
4,1506,2ae19629-1ca3-13a3-e055-000000000001,Y,STILLORGAN-MERVILLE,STIGH LORGAN-MERVILLE,267127,1,05,DUN LAOGHAIRE/RATHDOWN,Baile Átha Cliath,STILLORGAN,"POLYGON ((-6.19482 53.28856, -6.19454 53.28778..."
...,...,...,...,...,...,...,...,...,...,...,...,...
317,2087,2ae19629-1eef-13a3-e055-000000000001,Y,AIRPORT,AN TAERFORT,267001,1,04,FINGAL,Baile Átha Cliath,SWORDS,"POLYGON ((-6.24823 53.43693, -6.24783 53.43634..."
318,2088,2ae19629-1ef0-13a3-e055-000000000001,Y,BALBRIGGAN RURAL,BAILE BRIGÍN (TUATH),267002,1,04,FINGAL,Baile Átha Cliath,BALBRIGGAN,"POLYGON ((-6.21246 53.63361, -6.21237 53.63357..."
319,2089,2ae19629-1ef1-13a3-e055-000000000001,Y,BALBRIGGAN URBAN,BAILE BRIGÍN (UIRBEACH),267003,1,04,FINGAL,Baile Átha Cliath,BALBRIGGAN,"POLYGON ((-6.18305 53.61637, -6.18305 53.61637..."
320,2090,2ae19629-1ef2-13a3-e055-000000000001,Y,BALDOYLE,BAILE DÚILL,267004,1,04,FINGAL,Baile Átha Cliath,HOWTH-MALAHIDE,"POLYGON ((-6.13494 53.40693, -6.13485 53.40681..."
